In [ ]:
import scanpy as sc
import numpy as np
from typing import List
import scvelo as scv
import matplotlib.colors as clr
import matplotlib.pyplot as plt

adding the uninfected data to the time course data

In [ ]:
adata = sc.read_h5ad("uninfected/analysis/cleaned/final_object_no_peyers.h5ad")
replicates_adata = sc.read("timecourse.h5ad")
total_adata = sc.concat([adata, replicates_adata])

Adding a "timepoint" observation to the adata

In [ ]:
mapping = {
    "day6_SI": "D6",
    "day6_SI_r2": "D6",
    "day8_SI_Ctrl": "D8",
    "day8_SI_r2": "D8",
    "day30_SI": "D30",
    "day30_SI_r2": "D30",
    "day90_SI": "D90",
    "day90_SI_r2": "D90",
    "segmentation_SI1": "Uninfected",
    "segmentation_SI2": "Uninfected",
}

In [ ]:
maps = [mapping.get(i) for i in total_adata.obs["batch"]]
total_adata.obs["timepoint"] = maps

total_adata = total_adata[~total_adata.obs["timepoint"].isna()]
total_adata = total_adata[~total_adata.obs["Subtype"].isin(["Contaminated DCs"])]

Saving out dotplots of key gene expression

In [ ]:
for batch in np.unique(total_adata.obs["timepoint"]):
    fig = sc.pl.dotplot(
        total_adata[total_adata.obs["timepoint"] == batch],
        groupby="Subtype",
        var_names=[
            "Il18",
            "Il15",
            "Tgfb1",
            "Tgfb2",
            "Tgfb3",
            "Slamf6",
            "Cxcl9",
            "Cxcl10",
        ],
        return_fig=True,
        show=False,
        standard_scale="var",
        smallest_dot=20,
    )
    fig.savefig(f"../figures/dotplots/{batch}.pdf")
    plt.close()